In [ ]:
teed_filename = 'data/teed_full.csv'

In [ ]:
fontsize = 30
fontsize_ax = 22

In [ ]:
import pandas as pd

df = pd.read_csv(teed_filename, sep=',')

df.columns = ['filename', 'condition', 'rat', 'rat_group', 'stim_type', 'max_amplitude', 'power', 'power_absolute']
df = df.explode('power')
df['power'] = df['power'].astype('float')
df['power_absolute'] = df['power_absolute'].apply(lambda x: x / 1e9)

In [ ]:
print('ON/OFF: %.2f +- %.2f' % (df[df['stim_type'] == 'on-off']['power'].mean(), df[df['stim_type'] == 'on-off']['power'].std()))
print('Proportional: %.2f +- %.2f' % (df[df['stim_type'] == 'proportional']['power'].mean(), df[df['stim_type'] == 'proportional']['power'].std()))

In [ ]:
import matplotlib.pyplot as plt
import ratdata.plot as rdplot
import seaborn as sns

order_labels = ['on-off', 'proportional']
plt.figure(figsize=(10, 10))
ax = sns.violinplot(x='stim_type', y='power', data=df, order=order_labels, palette=rdplot.stim_type_palette, bw=0.2)
for violin in ax.collections[::2]:
    violin.set_alpha(rdplot.boxplot_alpha)
sns.swarmplot(x='stim_type', y='power', data=df, order=order_labels, s=8, palette=rdplot.stim_type_palette)
plt.ylim([0, 110])
plt.axhline(100, color='red')

ax.tick_params(axis='both', which='major', labelsize=fontsize_ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel('Stimulation type', fontsize=fontsize_ax)
plt.ylabel('Stimulation power compared to open-loop [%]', fontsize=fontsize_ax)
plt.title('Estimated stimulation power draw', fontsize=fontsize)
plt.savefig('plots/teed.svg', bbox_inches='tight', facecolor='white')
plt.savefig('plots/teed.png', bbox_inches='tight', facecolor='white')

In [ ]:
# import matplotlib

# font = {'family' : 'normal',
#         'weight': 'normal',
#         'size' : 20}
# matplotlib.rc('font', **font)

order_labels = ['continuous', 'on-off', 'proportional']
fig, axs = plt.subplots(1, 3, figsize=(22, 8), sharey=True)

i = 0
rats = df['rat'].unique()
rats.sort()
for rat in rats:
    if 'rat D' in rat:
        data = df[df['rat'] == rat]
        sns.violinplot(ax=axs[i], x='stim_type', y='power_absolute', data=data, order=order_labels, palette=rdplot.stim_type_palette, bw=0.2)
        for violin in axs[i].collections[::2]:
            violin.set_alpha(rdplot.boxplot_alpha)
        sns.swarmplot(ax=axs[i], x='stim_type', y='power_absolute', data=data, order=order_labels, s=8, palette=rdplot.stim_type_palette)        
        axs[i].set_xlabel('Stimulation type', fontsize=fontsize_ax, labelpad=30)
        axs[i].set_title('%s' % rat, fontsize=fontsize)
        axs[i].set_ylim([0, 0.12])
        axs[i].tick_params(axis='both', which='major', labelsize=fontsize_ax)
        axs[i].spines['top'].set_visible(False)
        axs[i].spines['right'].set_visible(False)
        i += 1
axs[0].set_ylabel('Power [a.u.]', fontsize=fontsize_ax)
axs[1].set_ylabel(None)
axs[2].set_ylabel(None)
fig.suptitle('Average stimulation power', fontsize=fontsize)
plt.subplots_adjust(wspace=0.05)
plt.savefig('plots/teed_by_rat.svg', bbox_inches='tight', facecolor='white')
plt.savefig('plots/teed_by_rat.png', bbox_inches='tight', facecolor='white')